In [92]:
%run helpers.ipynb


In [93]:
def gradient_descent(X,Y,X_val,Y_val,activation_func, loss_func, eta, num_epochs, num_hidden_layers, sizes, batch_size,inputsize,outputsize):
    
    print('Gradient_descent')
    train_loss_list=[]
    val_loss_list=[]
    #param_after_every_epoch={}
    epoch = 0
    params = createnetwork(num_hidden_layers, activation_func, sizes, inputsize, outputsize)
    #for every epoch
    while epoch < num_epochs:
        print(epoch)
        
        grads = creategrads(num_hidden_layers, sizes, inputsize,outputsize)
        #in every epoch for every sample
        sample_indices = gen_sample_indices(batch_size,X.shape[0])
        for j in sample_indices:
            x = X[j,:]  #taking jth sample
            
            y = Y[j,:]
            
            #forward propogation
            yhat, A, H = fwd_prop(x, params, activation_func, num_hidden_layers)
            
            #Backward propogation for gradients
            grad_current = back_prop(H,A, params, num_hidden_layers, sizes, y, yhat, loss_func, activation_func, inputsize, outputsize)
            
            #summing the grads
            #print(grad_current)
            for key in grads:
                grads[key] = grads[key] + grad_current[key]
                
            #all grads are summed for all samples now we need to update the parameters
            for newkey in params:
                params[newkey] = params[newkey] - eta * (grads["d"+newkey])
        
        #again restoring the grads dictionary for next epoch
        grads = creategrads(num_hidden_layers, sizes, inputsize, outputsize)
        
        #Finding accuracy and loss for every epoch
        
        
        
        train_err, train_loss, val_error, val_loss = measure_performance(X,Y, X_val, Y_val,params, activation_func, num_hidden_layers, loss_func)
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        print("for epoch {} train error: {}  train loss:{}  validation error:{}  validation loss:{}".format(epoch, train_err, train_loss, val_error, val_loss))
        #print(params)
        epoch +=1
        
    return train_loss_list, val_loss_list


In [94]:
def minibatch_gradient_descent(X,Y,X_val,Y_val,activation_func, loss_func, eta, num_epochs, num_hidden_layers, sizes, batch_size,inputsize,outputsize):
    
    print('Minibatch_Gradient_descent with batch size {}'.format(batch_size))
    
    #param_after_every_epoch={}
    epoch = 0
    epochs=[]
    train_loss_list=[]
    val_loss_list=[]
    params = createnetwork(num_hidden_layers, activation_func, sizes, inputsize, outputsize)
    #for every epoch
    while epoch < num_epochs:
        print(epoch)
        pointsseen=0
        grads = creategrads(num_hidden_layers, sizes, inputsize,outputsize)
        #in every epoch for every sample
        sample_indices = gen_sample_indices(batch_size,X.shape[0])
        for j in sample_indices:
            x = X[j,:]  #taking jth sample
            
            y = Y[j,:]
            
            #forward propogation
            yhat, A, H = fwd_prop(x, params, activation_func, num_hidden_layers)
            
            #Backward propogation for gradients
            grad_current = back_prop(H,A, params, num_hidden_layers, sizes, y, yhat, loss_func, activation_func, inputsize, outputsize)
            
            #summing the grads
            #print(grad_current)
            for key in grads:
                grads[key] = grads[key] + grad_current[key]
            pointsseen+=1    
            #all grads are summed for all samples now we need to update the parameters
            if(pointsseen%batch_size == 0):
                for newkey in params:
                    params[newkey] = params[newkey] - eta * (grads["d"+newkey])
        
        #again restoring the grads dictionary for next epoch
        grads = creategrads(num_hidden_layers, sizes, inputsize, outputsize)
        
        #Finding accuracy and loss for every epoch
        
        
        
        train_err, train_loss, val_error, val_loss = measure_performance(X,Y, X_val, Y_val,params, activation_func, num_hidden_layers, loss_func)
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        epochs.append(epoch)
        print("for epoch {} train error: {}  train loss:{}  validation error:{}  validation loss:{}".format(epoch, train_err, train_loss, val_error, val_loss))
        #print(params)
        epoch +=1
        
    return train_loss_list, val_loss_list,epochs


In [95]:
def mgd(X, Y, X_val, Y_val, activation_func, loss_func, eta, gamma, num_epochs, num_hidden, sizes, batch_size,
 inputsize, outputsize):
    print("momentum gradient descent")
    # data for information display and plots
    step_data = {}
    epoch_data = []
    train_loss_list=[]
    val_loss_list=[]
    epochs =[]

    params = createnetwork(num_hidden, activation_func, sizes, inputsize, outputsize)



    prev_momenta = createmomenta(num_hidden, sizes, inputsize, outputsize)
    momenta = createmomenta(num_hidden, sizes, inputsize, outputsize)
    pointsseen = 0

    epoch = 0
    while epoch < num_epochs:
        grads = creategrads(num_hidden, sizes, inputsize, outputsize)
        step = 0
        sample_indices = gen_sample_indices(batch_size,X.shape[0])
        for j in sample_indices:
            x = X[j,:]
            y = Y[j,:]
            yhat, A, H = fwd_prop(x, params, activation_func, num_hidden)
            grad_current = back_prop(H, A, params, num_hidden, sizes, y, yhat, loss_func, activation_func, inputsize, outputsize)
            for key in grads:
                grads[key] = grads[key] + grad_current[key]

            pointsseen = pointsseen + 1

            # check if one step is done, update parameters, initialize new gradients
            if pointsseen% batch_size == 0:
                for newkey in params:
            # TODO: (0) add batch_size wala division
                    momenta["v" + newkey] = gamma*prev_momenta["v" + newkey] + eta*grads["d" + newkey]
                    params[newkey] = params[newkey] - momenta["v" + newkey]
                    prev_momenta["v" + newkey] = momenta["v" + newkey]
                grads = creategrads(num_hidden, sizes, inputsize, outputsize)

                step = step + 1
                # store data for log files if 100 steps are done
        train_err, train_loss, val_error, val_loss = measure_performance(X, Y, X_val, Y_val, params, activation_func, num_hidden, loss_func)
        print("for epoch {} train error: {}  train loss:{}  validation error:{}  validation loss:{}".format(epoch, train_err, train_loss, val_error, val_loss))
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        epochs.append(epoch)
        epoch = epoch + 1

    return train_loss_list, val_loss_list, epochs


In [96]:
def adam(X, Y, X_val, Y_val, activation_func, loss_func, eta, num_epochs, num_hidden, sizes,batch_size,inputsize, outputsize, beta1, beta2, eps):
    print('Adam Optimization')
    
    train_loss_list=[]
    val_loss_list=[]
    epochs_list=[]
    
    params=createnetwork(num_hidden, activation_func, sizes, inputsize, outputsize)
    
    prev_momenta = createmomenta(num_hidden, sizes, inputsize, outputsize)
    momenta = createmomenta(num_hidden, sizes, inputsize, outputsize)
    momenta_hat = createmomenta(num_hidden, sizes, inputsize, outputsize)
    
    prev_momenta_squared = createmomenta_squared(num_hidden, sizes, inputsize, outputsize)
    momenta_squared = createmomenta_squared(num_hidden, sizes, inputsize, outputsize)
    momenta_squared_hat = createmomenta_squared(num_hidden, sizes, inputsize, outputsize)
    
    pointsseen = 0
    epoch = 0
    
    while epoch < num_epochs:
        step=0
        grads = creategrads(num_hidden, sizes, inputsize, outputsize)
        
        sample_indices = gen_sample_indices(batch_size,X.shape[0])
        for j in sample_indices:  #TODO: Problem here is at every epoch we are selecting same hundered epochs
            x=X[j,:]   #TODO: even if we run this loop for X.shape[0] we are just considering the parameters updated after 100 samples
            y=Y[j,:]   #TODO: generate 100 random integers for every epoch in the range of 0 - X.shape[0] and run for those idices (100 times) and update parameters
            
            yhat, A, H = fwd_prop(x, params, activation_func, num_hidden)
            
            grad_current = back_prop(H, A, params, num_hidden, sizes, y, yhat, loss_func, activation_func, inputsize, outputsize)
            
            for key in grads:
                grads[key] = grads[key]+grad_current[key]
                
            pointsseen+=1
            
            if pointsseen%batch_size == 0:
                step =step+1
                for newkey in params:
                    momenta["v"+newkey] = beta1*prev_momenta["v"+newkey]+(1-beta1)*grads["d"+newkey]
                    momenta_squared["m"+newkey] = beta2*prev_momenta_squared["m"+newkey]+(1-beta2)*((grads["d"+newkey])**2)
                    
                    momenta_hat["v" + newkey] = momenta["v" + newkey]/(1 - np.power(beta1, step))
                    momenta_squared_hat["m" + newkey] = momenta_squared["m" + newkey]/(1 - np.power(beta2, step))

                    params[newkey] = params[newkey] - (eta/np.sqrt(momenta_squared_hat["m" + newkey] + eps))*momenta_hat["v" + newkey]

                    prev_momenta["v" + newkey] = momenta["v" + newkey]
                    prev_momenta_squared["m" + newkey] = momenta_squared["m" + newkey]
                
                grads = creategrads(num_hidden, sizes, inputsize, outputsize)
                
        train_err, train_loss, val_err, val_loss= measure_performance(X, Y, X_val, Y_val,params, activation_func, num_hidden, loss_func)
        print("for epoch {} train error: {}  train loss:{}  validation error:{}  validation loss:{}".format(epoch, train_err, train_loss, val_err, val_loss))
        train_loss_list.append(train_loss)
        
        val_loss_list.append(val_loss)
        epochs_list.append(epoch)
        epoch+=1
    return train_loss_list, val_loss_list, epochs_list

In [97]:
def nag(X, Y, X_val, Y_val, activation_func, loss_func, eta, gamma, num_epochs, num_hidden, sizes, batch_size,
 inputsize, outputsize):
    print("NAG")
    epoch_list = []
    train_loss_list=[]
    val_loss_list=[]

    # initializing

    params = createnetwork(num_hidden, activation_func, sizes, inputsize, outputsize)

    prev_momenta = createmomenta(num_hidden, sizes, inputsize, outputsize)
    momenta = createmomenta(num_hidden, sizes, inputsize, outputsize)

    pointsseen = 0
    epoch = 0
    while epoch < (num_epochs):
        grads = creategrads(num_hidden, sizes, inputsize, outputsize)
        step = 0
        sample_indices = gen_sample_indices(batch_size,X.shape[0])
        for j in sample_indices:
            x = X[j,:]
            y = Y[j,:]
            yhat, A, H = fwd_prop(x, params, activation_func, num_hidden)
            grad_current = back_prop(H, A, params, num_hidden, sizes, y, yhat, loss_func, activation_func, inputsize, outputsize)
            for key in grads:
                grads[key] = grads[key] + grad_current[key]

            pointsseen = pointsseen + 1

            # check if one step is done, update parameters, initialize new gradients
            if pointsseen% batch_size == 0:
                step = step + 1

                for newkey in params:
            # TODO: (0) add batch_size wala division
                    momenta["v" + newkey] = gamma*prev_momenta["v" + newkey] + eta*grads["d" + newkey]
                    params[newkey] = params[newkey] - momenta["v" + newkey]
                    prev_momenta["v" + newkey] = momenta["v" + newkey]



                grads = creategrads(num_hidden, sizes, inputsize, outputsize)
                
                # TODO: review what happens in the last step
                for next_key in params:
                    momenta["v" + next_key] = gamma*prev_momenta["v" + next_key]
                    params[next_key] = params[next_key] - momenta["v" + next_key]

        train_err, train_loss, val_err, val_loss = measure_performance(X, Y, X_val, Y_val, params, activation_func, num_hidden, loss_func)
        print("for epoch {} train error: {}  train loss:{}  validation error:{}  validation loss:{}".format(epoch, train_err, train_loss, val_err, val_loss))
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        epoch_list.append(epoch)


        epoch = epoch + 1
    return train_loss_list, val_loss_list, epoch_list

